In [1]:
import finlab
finlab.login(api_token='ID0hv1JVLP3bs86Qg/xslT1fwKjN0bclQVRCH7eQiAYh3e5/XNh77u3zaPDOOGJQ#vip_m')
from finlab import data
from finlab.backtest import sim
import pandas as pd
import plotly
import matplotlib

輸入成功!


In [2]:
with data.universe(market ='STOCK_FUTURE'):
    benchmark = data.get('benchmark_return:發行量加權股價報酬指數', save_to_storage=True)
    close = data.get('price:收盤價', save_to_storage=True)
    volume = data.get('price:成交股數', save_to_storage=True)
    monthly_rev = data.get('monthly_revenue:當月營收', save_to_storage = True)	
    stock_fundamental = data.get('company_basic_info', save_to_storage = True)	
    monthly_rev_YOY = data.get('monthly_revenue:去年同月增減(%)')	

 Use "pip install finlab==0.4.4" to update the latest version.


In [3]:
monthly_rev

,1101,1102,1210,1216,1301,1303,1312,1314,1319,1326,...,8299,8358,8436,8454,8942,9904,9914,9938,9939,9945
date,,,,,,,,,,,,,,,,,,,,,
2005-M01,2746766.0,1094966.0,1426905.0,3194044.0,11436734.0,15485742.0,1108005.0,2372058.0,575130.0,14000837.0,...,419428.0,NaN,NaN,NaN,100527.0,980457.0,517149.0,94531.0,199859.0,729712.0
2005-M02,1566092.0,663833.0,1137814.0,2454186.0,10021713.0,12720633.0,1026112.0,2175971.0,464410.0,12892265.0,...,358257.0,NaN,NaN,NaN,60919.0,865100.0,459377.0,63210.0,138862.0,729620.0
2005-M03,2504657.0,968271.0,1361315.0,3044984.0,11676460.0,15715321.0,1366258.0,2960911.0,760475.0,13411900.0,...,510076.0,NaN,NaN,NaN,95654.0,1344990.0,458903.0,177676.0,199132.0,745498.0
2005-M04,2418388.0,876647.0,1330105.0,3052151.0,11083995.0,13855541.0,1082931.0,2994717.0,685288.0,12677312.0,...,439148.0,NaN,NaN,NaN,100408.0,1104505.0,401346.0,170801.0,246305.0,498365.0
2005-M05,2394229.0,934672.0,1297087.0,3400487.0,9937769.0,13094744.0,870891.0,2407876.0,660248.0,12398134.0,...,449312.0,NaN,NaN,NaN,90628.0,1008493.0,354763.0,172523.0,255782.0,579833.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-M01,7325221.0,4814494.0,9191048.0,45915517.0,14408136.0,20289828.0,1285657.0,1141581.0,1498170.0,25576976.0,...,2878167.0,621687.0,661903.0,8752352.0,350042.0,24801089.0,2284346.0,779389.0,1907806.0,1671450.0
2023-M02,7306069.0,6241236.0,8240329.0,40664831.0,17688186.0,21842845.0,1320956.0,1671640.0,1931211.0,28017455.0,...,3274873.0,582581.0,561018.0,7899181.0,508653.0,18231389.0,2699606.0,1069405.0,1914494.0,2778101.0
2023-M03,11730367.0,8152982.0,9633956.0,44823352.0,20116995.0,23641103.0,1610657.0,2332203.0,2168884.0,29434395.0,...,3925381.0,548588.0,649936.0,8468507.0,620350.0,21333920.0,3373877.0,1139724.0,2244120.0,1836194.0


In [4]:
monthly_rev_mom = (monthly_rev/monthly_rev.shift(1)) -1 
monthly_rev_yoy = (monthly_rev/monthly_rev.shift(12)) -1 

In [5]:
all_rev = data.get('monthly_revenue:前期比較增減(%)')	

In [6]:
open = data.get('price:開盤價')	

In [15]:
condition1 = monthly_rev_mom > monthly_rev_mom.shift(1).rolling(9).mean()
condition2 = monthly_rev_yoy > monthly_rev_yoy.shift(1).rolling(4).mean()

condition3 = monthly_rev_mom < monthly_rev_mom.shift(1).rolling(9).mean()
condition4 = monthly_rev_yoy < monthly_rev_yoy.shift(1).rolling(4).mean()
condition5 = monthly_rev_yoy < monthly_rev_yoy.shift(1)
condition6 = monthly_rev_mom < monthly_rev_mom.shift(1)

buy  = ((condition1 & condition2 & (close<40))*monthly_rev_yoy).is_largest(5)
sell = condition5 & condition6

In [17]:
buy = buy[buy.index >= '2018-01-01']


In [18]:
#  , nstocks_limit=20, rank=invest_trust_surplus
# position= buy.hold_until(sell, nstocks_limit=10, rank=monthly_rev_yoy)
report = sim(position=buy, resample='Q', trade_at_price='close', stop_loss=0.1, position_limit=0.2, fee_ratio=-1.425/1000*6,name="月營收均線交叉", live_performance_start='2023-02-10', mae_mfe_window=120, upload=False)
print(report.get_stats(resample='1d', riskfree_rate=0.02)['daily_sharpe'])
print(report.get_stats(resample='1d', riskfree_rate=0.02)['max_drawdown'])
report.display()
report.display_mae_mfe_analysis()

1.6645300123852564
-0.14949170148238777


--Stats--
annualized_rate_of_return     33.28%
sharpe                          1.66
max_drawdown                 -14.95%
win_ratio                     56.38%

--Latest Position--


1101    0.2
2485    0.2
2515    0.2
2887    0.2
6153    0.2
Name: 2023-06-30 00:00:00, dtype: float64

In [10]:
report.trades

,stock_id,entry_date,exit_date,entry_sig_date,exit_sig_date,position,period,entry_index,exit_index,return,trade_price@entry_date,trade_price@exit_date,mae,gmfe,bmfe,mdd,pdays
trade_index,,,,,,,,,,,,,,,,,
0,1718 中纖,2019-01-02,2019-04-01,2018-12-28,2019-03-29,0.066667,54.0,2899.0,2953.0,-0.004975,10.05,10.00,-0.039801,0.014925,0.014925,-0.053922,11.0
1,2388 威盛,2019-01-02,2019-04-01,2018-12-28,2019-03-29,0.066667,54.0,2899.0,2953.0,0.510597,25.95,39.20,-0.082852,0.541426,0.000000,-0.082852,35.0
2,2449 京元電子,2019-01-02,2019-04-01,2018-12-28,2019-03-29,0.066667,54.0,2899.0,2953.0,0.158351,23.05,26.70,-0.073753,0.158351,0.000000,-0.073753,37.0
3,2515 中工,2019-01-02,2019-04-01,2018-12-28,2019-03-29,0.066667,54.0,2899.0,2953.0,0.090780,7.05,7.69,-0.002837,0.148936,0.000000,-0.050617,52.0
4,2520 冠德,2019-01-02,2019-04-01,2018-12-28,2019-03-29,0.066667,54.0,2899.0,2953.0,-0.031175,20.85,20.20,-0.088729,0.023981,0.023981,-0.110070,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,2485 兆赫,NaT,NaT,2023-06-30,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,2887 台新金,NaT,NaT,2023-06-30,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,2890 永豐金,NaT,NaT,2023-06-30,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
report.trades['return'].max()

5.600000000000001